In [28]:
import requests
from cryptography.hazmat.primitives import serialization
from lib.utils import *

def get_public_key_pem(public_key):
    return public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

session = requests.Session()

# 请求服务器的公钥
response = session.get("http://localhost:8080/public_key")
server_public_key = serialization.load_pem_public_key(response.content)

# 生成对称密钥并加密
symmetric_key = generate_symmetric_key()

# 使用公钥对象获取 PEM 格式的公钥
server_public_key_pem = get_public_key_pem(server_public_key)

# 使用 PEM 格式的公钥进行加密
encrypted_symmetric_key = encrypt_msg_with_public_key(symmetric_key, server_public_key_pem)

# 发送加密的对称密钥给服务器
session.post("http://localhost:8080/receive_symmetric_key", data=encrypted_symmetric_key)

# 使用对称密钥加密通信内容
data = b"Secret message"
encrypted_data = symmetric_encrypt_msg(data, symmetric_key)
# 发送加密数据
response = session.post("http://localhost:8080/encrypted_endpoint", data=encrypted_data)

# 解密返回的数据
encrypted_response = response.content
decrypted_response = symmetric_decrypt_msg(encrypted_response, symmetric_key)

# 处理或打印解密后的响应
print("Origin request:", data)
print("Encrypted request:", encrypted_data)
print("Origin response:", encrypted_response)
print("Decrypted response:", decrypted_response.decode())


Origin request: b'Secret message'
Encrypted request: b'gAAAAABlksDV5JFs7TZz_DVmAvxwIbvVX1awUW1gfcoaQlZNXxM-VRhcUA6mIOVzMLvGCkkTmhwGwmU00PBI3qglSNq-Jm7ibA=='
Origin response: b'gAAAAABlksDVh4HXxCyjza2d_x6mc8h0vJLC9QIuDdHARODvwMaIueg46sqdijDyHY9gxN2WxlAFYFO3n_gC9PmRJ6igkIeP5q7kRL9-URlDQiwF2ZjXAQQ='
Decrypted response: Response to encrypted request


In [29]:
import logging

# 定义自定义日志格式和颜色
class CustomFormatter(logging.Formatter):
    """自定义一个日志格式器"""

    YELLOW = '\033[33m'
    WHITE = '\033[37m'
    RESET = '\033[0m'

    FORMAT = "%(levelcolor)s%(levelname)s%(reset)s: %(messagecolor)s%(message)s%(reset)s"

    FORMATS = {
        logging.WARNING: FORMAT.replace('%(levelcolor)s', YELLOW).replace('%(messagecolor)s', WHITE)
    }

    def format(self, record):
        log_fmt = self.FORMATS.get(record.levelno)
        formatter = logging.Formatter(log_fmt, style='%')
        formatter.default_time_format = '%Y-%m-%d %H:%M:%S'
        formatter.default_msec_format = '%s.%03d'
        record.reset = self.RESET
        return formatter.format(record)

def setup_logger():
    """设置日志记录器"""
    logger = logging.getLogger("custom_logger")
    logger.setLevel(logging.WARNING)
    ch = logging.StreamHandler()
    ch.setLevel(logging.WARNING)
    ch.setFormatter(CustomFormatter())
    logger.addHandler(ch)
    return logger

# 使用自定义日志记录器
logger = setup_logger()

# 测试日志记录器
logger.warning("This is a warning message with %s", "variable parameters")
